# MP 3: Grasp Planning

**Due date**: March 7, 2022 at 9:45am.

**Instructions**: Read and complete the problems below. In this assignment, you should be switched over to a local install.  As a backup, you may run code through this notebook via Jupyter Notebook or via Binder, but be aware that you will be editing external .py files, and saving your progress is not as simple as clicking "Save notebook state" in this window.

Your code for Problem X will go inside `problemX.py`.  For a local install, just run `python problemX.py` and use the actions in the menu to test your code. For Jupyter / Binder, run the cells at the end of this notebook.

To submit your assignment, perform the following:

1. Double-check that your programs run without error.
2. Send this file, all of your .py files, .json files, and any other files you used in your programs on Moodle [http:/learn.illinois.edu](http:/learn.illinois.edu).
3. If you are using any external libraries other than the ones that are indicated during the installation process, include a README file indicating which library and version you are using.  To be on the safe side, you should include a backup procedure until the graders verify that they are able to support use of that library.

## Problem 1: Grasp approach planning

A. Write a subroutine that will generate keyframes for an approach motion for the gripper to attain a grasp. The gripper should start at a finger-open configuration, move forward along its local `primaryAxis` by `distance` units, and then close its gripper so its fingers are at width `grasp.finger_width`. The return value is a movement of the gripper's base link and its finger links stored separately.  The finger configuration at varying opening amounts can be obtained using `GripperInfo.partwayOpenConfig(u)` with `u` in the range $[0,1]$.

The finger trajectory will be returned as a `Trajectory` and the gripper base trajectory will be an `SE3Trajectory` classes. The trajectories consist of a sequence of times `[t0,...,tn]` and  milestones (keyframes) `[m0,...,mn]`, which are then linearly interpolated for you using the `eval()` method.  SE3Trajectory differs from Trajectory in that the interpolation is done in SE3, so that if you do perform rotations during the approach motion, they will be interpolated properly with geodesics. Create it using the constructor `SE3Trajectory(times,Ts)` where `Ts` is a list of klampt se3 objects `[T0,...,Tn]`.

If you want to visually debug your outputs or manually craft a trajectory, you may uncomment out the block beginning with `resource.edit`.  (This is only available on a local install, and may not work properly on Mac)

*Note*: in this assignment we will be using AntipodalGrasp "databases" which are stored on disk in JSON format. A database for the hammer object is given to you. If you would like, you can reuse your code from MP2 to generate new grasp databases, making sure to use the code in `antipodal_grasp.py` to save them in the proper format.  Note that we have added the `score` attribute which should be used to store the score.  Code to do this, if you're running in `MP2.ipynb`, is as follows.

```
if sys.path[-1] != "../MP3":
    sys.path.append("../MP3")
import antipodal_grasp

grasps = antipodal_grasp_sample_surface(known_grippers.robotiq_140,obj,50,2000)
grasps_to_save = []
for g,s in grasps:
    ag = antipodal_grasp.AntipodalGrasp(g.center,g.axis)
    ag.finger_width = g.finger_width
    ag.approach = g.approach
    ag.contact1 = g.contact1
    ag.contact2 = g.contact2
    ag.score = s
    grasps_to_save.append(ag)
antipodal_grasp.save_antipodal_grasp_database(grasps_to_save,"048_hammer.json")
```


## Problem 2: Inverse kinematics

In this problem, you will find a configuration for a robot that meets a grasp by solving an inverse kinematics problem. We will use the TRINA robot and the gripper on its left arm for these problems. 

A. The `Sample Transform + IK` function will pick a transform at random, and then call your `solve_robot_ik` function. Implement an inverse kinematics solver using the [klampt.model.ik](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/klampt.model.ik.html#module-klampt.model.ik) functions. You may also need to consult the [IKSolver](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/klampt.robotsim.html?highlight=iksolver#klampt.IKSolver) docs and the [IK manual](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/Manual-IK.html).  If the solver succeeds, the gripper will be colored green.  

Make sure to configure the active degrees of freedom to avoid altering the robot's links 0-5.  The indices of the links of the left arm that should move are 11-16 (inclusive).

B. Observe that many sampled transforms are not reached, either because the transform is unreachable, or the solver simply fell into a local minimum.  The "sample-transform-then-IK" approach fails to take into account that by freezing the transform, the first step causes possible failures in the second step.  As an alternative, you will implement a solver that directly captures the antipodal grasp's axis rotation constraint.  In `sample_grasp_ik`, create a solver that solves an IK problem with a hinge-type constraint. 

Don't forget to calculate the gripper link's transform as the second return value.

**Written.** Which method is more efficient at generating successful grasping configurations?  Do you think that this will generally be the case? Why or why not? 

C. Create a new function `solve_grasp_ik` that only returns successfully if the resulting configuration is collision-free with respect to the robot and the object, only. (Ignore collisions between the object and the finger links, which are named `left_gripper:right_inner_finger_pad` and `left_gripper:left_inner_finger_pad`).  The `Geometry3D.collides` function and the `RobotModel.selfCollides` function will be helpful here.  

Furthermore, implement a random-restart technique with 20 restarts to increase the likelihood of sampling a collision-free, IK-solving configuration.

D. If the robot is asked to grasp an object in the presence of obstacles, it needs to perform more sophisticated reasoning about collisions, and possibly explore the use of different grasps.  In `solve_grasp_problem`, implement a grasp planner that checks amongst multiple grasps, and outputs a grasp and IK configuration such that the grasp has a relatively good (low) score and  such that the configuration is collision free with respect to the object and all obstacles (such as the table and box).

**Written.** How long does the grasp solver take in practice? Describe how the following parameters would be expected to affect solution times:
- the number of grasps in the database,
- the number of IK solver iterations,
- the number of restarts in the IK solver,
- the location of the target object relative to the arm's workspace, and
- the amount of clutter (obstacles) around the target object.


## Problem 3: Motion planning

Finally, we will look at how to combine the approach path, IK, and motion planning to produce a complete robot trajectory that is very likely to pick the object when executed.

A. Implement a path planner that uses the [Klampt motion planning API](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/Manual-Planning.html) in the `feasible_plan` function.  The "standard" API requires you to configure a [MotionPlan](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/klampt.plan.cspace.html#klampt.plan.cspace.MotionPlan) object and then repeatedly call `MotionPlan.planMore()` until a desired termination criterion has been reached.

During the setup phase, you will need to determine:
* the active degrees of freedom
* the relevant collision tests to perform
* the edge collision checking resolution
* the planner type
* other planner parameters, in particular the connection radius.

In your first implementation, use only the 6 degrees of freedom of the left arm; perform collision tests between every link and every other object; use a collision checking resolution of 0.01 (radians); use the SBL planner; and set the planner parameters perturbationRadius=0.5 and connectionThreshold=3.0.

During the planning phase, you will need to control how many iterations to perform. In your implementation, run 1 iteration on each loop until 5 seconds have elapsed, or a plan has been found.  (Monitor the system time using the Python `time.time()` function)

Every time you choose the "Plan" action, your function will be called to find a plan to the target, which is edited using the GUI.  Make sure to test your planner on both easy and hard queries.

B. **4-credit section only**. Explore the use of optimizing motion planners. Replace `feasible_plan` with `optimizing_plan` and implement an optimizing planner.  SBL with shortcutting and/or restarts, RRT with shortcutting and/or restarts, Lazy-RRT*, or Lazy-PRM* are reasonable options.

C. You will now integrate all these pieces together into a full grasp plan.  In `plan_grasping_motion`, you will 1) sample a collision-free grasp configuration, 2) determine the gripper approach trajectory, 3) determine the robot configuration at the start of the approach trajectory, using IK, 4) plan a collision-free path to the start of the approach, and 5) convert the gripper approach trajectory into a trajectory for the full robot.  Perform error checking to fail gracefully (i.e., return (None,None) ) when any piece of this procedure fails.

*This is a fairly intricate procedure with many moving pieces, and there are many places where your code can go wrong.  Don't save this until the last minute!*

**Written.** You might notice that the resulting path is quite suboptimal and is likely to twirl the end effector around, even if you are using `optimizing_plan` to plan your path. Explain why the structure of this 5-step procedure could be causing the problem. In a few sentences, sketch out a possible remedy to this problem (no need to implement it).

## Jupyter cells for non-local installs

In [1]:
%load_ext autoreload
%autoreload 1
from klampt import vis
vis.init('IPython')
closeup_viewport = closeup_viewport = {'up': {'z': 0, 'y': 1, 'x': 0}, 'target': {'z': 0, 'y': 0, 'x': 0}, 'near': 0.1, 'position': {'z': 1.0, 'y': 0.5, 'x': 0.0}, 'far': 1000}

In [ ]:
## Problem 1: 
%aimport problem1

vis.createWindow()
vis.resizeWindow(600,400)
vis.setViewport(closeup_viewport)
problem1.problem_1()

In [ ]:
## Problem 2 
%aimport problem2

vis.createWindow()
vis.resizeWindow(600,400)
problem2.problem_2()

In [2]:
## Problem 3.A or B
%aimport problem3

vis.createWindow()
vis.resizeWindow(600,400)
problem3.problem_3ab()

KlamptWidgetAdaptor(height=400, rpc={'type': 'multiple', 'calls': [{'type': 'add_ghost', 'object': 'TRINA', 'p…

Playback(children=(HBox(children=(Button(description='Play', icon='play', style=ButtonStyle(), tooltip='Start …

Planned path with 2 milestones, length 0.0
Planned path with 2 milestones, length 1.339000000000001


In [ ]:
## Problem 3.C
%aimport problem3

vis.createWindow()
vis.resizeWindow(600,400)
problem3.problem_3ab()

## Written answers

### 2.B
Place your answers to the written portion of 2.B here

### 2.D
Place your answers to the written portion of 2.D here

### 3.C
Place your answers to the written portion of 3.C here
